#  Electricity Price Forecasting – Comparative Modeling Project

**Project Goal:** Develop and compare statistical, machine learning, and deep learning models to forecast hourly electricity prices one-step ahead and 24-steps ahead.

**Dataset Period:** 2015–2018 (4 years of hourly electricity prices, energy generation, and weather data)

**Methodology:** CRISP-DM (Cross-Industry Standard Process for Data Mining)

---

## 1. Business Understanding

### 1.1 Business Objectives
Electricity is a critical resource for modern societies, and its price directly affects consumers, producers, and energy market participants. With the liberalization of electricity markets, prices fluctuate on an hourly basis depending on supply, demand, weather conditions, and generation sources. These fluctuations introduce uncertainty for grid operators, energy traders, and policy makers, making accurate price forecasting an essential task.

The objective of this project is to develop and compare different forecasting models capable of predicting electricity prices one hour ahead and one day ahead (24 hours). By leveraging historical electricity prices together with exogenous variables related to energy generation and weather conditions, the models aim to capture both short-term dynamics and daily seasonal patterns. Accurate forecasts can support better decision-making in energy trading, grid management, and operational planning, ultimately contributing to market efficiency and system stability.

### 1.2 Project Goals and Successs Criteria
From a data mining perspective, this problem is formulated as a time-series regression task, where the target variable is the electricity price at future time steps. The models are trained using four years of hourly data (2015–2018), with the final 12 months reserved as an out-of-sample test set to realistically evaluate forecasting performance.

The main goal of the project is not only to produce accurate forecasts, but also to compare different modelling paradigms, including statistical models, machine learning models, and deep learning models, under the same experimental setup. This comparative analysis allows us to assess the trade-offs between accuracy, stability, complexity, and interpretability.

Model performance is evaluated using standard regression error metrics: Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE). Additionally, all models are compared against a Seasonal Naïve baseline to ensure that improvements are meaningful and not trivial.

The success criteria for this project are defined as follows:
- Achieving a clear improvement over the Seasonal Naïve baseline for both 1-step and 24-step forecasts
- Maintaining stable performance across time-series cross-validation folds
- Producing forecasts that follow the main temporal patterns and seasonality of electricity prices
- Balancing predictive performance with computational efficiency, especially for multi-step forecasting
- Ensuring that the selected model is practical and interpretable enough to be used in real-world energy forecasting scenarios

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import clone
from sklearn.svm import SVR
from sklearn.ensemble import (
    BaggingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from statsmodels.tsa.seasonal import STL
import math
from statsmodels.nonparametric.smoothers_lowess import lowess
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as stats
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from statsmodels.nonparametric.smoothers_lowess import lowess


---
## 2. Data Understanding

### 2.1 Data Collection

The first step is to load the datasets into the working environment. This involves importing the necessary libraries and reading the data file into a suitable data structure, namely, a DataFrame using Pandas.

In [ ]:
# Load datasets
weather = pd.read_csv("weather.csv", parse_dates=["time"])
energy = pd.read_csv("energy_dataset.csv", parse_dates=["time"])

# Set 'time' as index
weather = weather.set_index("time")
energy = energy.set_index("time")

### 2.2 Data Description

##### Basic dataset information

In [ ]:
energy.info()

The *energy* dataset contains 35064 entries and 24 features, representing hourly records of electricity generation from various sources, total system load, and the day-ahead market price. Each row corresponds to one hour of energy system operation, and the goal is to forecast the electricity price one hour and one day ahead.

In [ ]:
energy.describe()

In [ ]:
energy.head()

In [ ]:
weather.info()

The *weather* dataset also contains 35064 entries and 5 features representing hourly meteorological measurements such as temperature, pressure, humidity, and wind speed. Each row corresponds to one hour of weather conditions, and these variables are used as exogenous inputs to improve electricity price forecasting.

In [ ]:
weather.describe()

In [ ]:
weather.head()

### 2.3 Data Exploration

#### 2.3.1 Target variable analysis

The target variable `price_day_ahead` represents the eletricity market price for the upcoming hour.

In [ ]:
target_col='price_day_ahead'

plt.figure(figsize=(8,5))
plt.hist(energy[target_col], bins=50, edgecolor='black')
plt.title("Histogram of Day-Ahead Electricity Price")
plt.xlabel("Price (€/MWh)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

The histogram shows a distribution that is very close to a normal distribution, although it is slightly right-skewed.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(energy[target_col])
plt.title("Time Series of Electricity Price (Day-Ahead)")
plt.xlabel("Time")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()

The time series plot shows strong short-term fluctuations and clear seasonal patterns, with occasional price spikes. Prices vary over time, indicating non-stationarity and the presence of both volatility and periodic behavior.

#### 2.3.2 Feature distribution analysis

Now we will perform feature distribution analysis to examine how the data values are spread across the datasets. We will use plots and histograms to visualise the distributions features. Box plots for will be skipped in this step, as they will be specifically used for outlier detection in paragraph `2.4.2`.

In [ ]:
def feat_distribution(nonDiscreteFeatures, df):
    nrows = math.ceil(len(nonDiscreteFeatures) / 2)
    ncols = 2
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, nrows * 5))
    axes = axes.flatten()

    for i, col in enumerate(nonDiscreteFeatures):
        df[col].hist(bins=30, ax=axes[i])
        axes[i].set_title(f'{col} Distribution')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')

    plt.show()

All of the features are numerical and continuous.

##### Energy dataset

In [ ]:
energy_features = [col for col in energy.columns 
                   if col not in ['time', 'price_day_ahead']]
feat_distribution(energy_features, energy)

None of the features has a normal distribution. `generation_hydro_pumped_storage_consumption`, `generation_hydro_water_reservoir`, and `generation_solar` are strongly right-skewed, while `generation_wind_onshore` is moderately right-skewed. In contrast, `generation_waste` is left-skewed.
Variables with skewed distributions will be handled in paragraph `3.4.3`, because skewness can negatively affect the models used later in paragraph `4.2`, particularly KNN, SVR, and Linear Regression, which rely on distance metrics or assumptions of linearity.

`generation_marine`, `generation_geothermal`, `generation_fossil_oil_shale`, `generation_fossil_peat`, and `generation_fossil_coal-derived_gas` contain only zeros, which is consistent with the dataset description provided in paragraph `2.2`.

The data represented by the remaining variables generally show irregular, multimodal, or heavily skewed shapes, reflecting the diverse and highly variable nature of electricity generation across different sources.

##### Weather dataset

In [ ]:
weather_features = [col for col in weather.columns if col != 'time']
feat_distribution(weather_features, weather)

`temperature` and `pressure` distribution is close to normal, whereas `humidity` is slightly left-skewed and `wind_speed` is right-skewed.

Skeweness will be handled in paragraph `3.3.1`.

In [ ]:
skewed_cols = ['generation_hydro_pumped_storage_consumption','generation_hydro_water_reservoir','generation_solar','generation_wind_onshore','generation_waste','wind_speed']

### 2.4 Data Quality Assessment

#### 2.4.1 Identify missing values

In the first place, we will check for missing values to ensure data completeness and avoid potential issues during analysis and modeling.

In [ ]:
# % missing values by column

def missing_values_info(df):
    nulls = df.isnull().sum()
    percent = round(nulls / df.shape[0] * 100, 3)
    
    nullvalues = pd.concat([nulls, percent], axis=1)
    nullvalues.columns = ["Count", "%"]
    
    return nullvalues


In [ ]:
missing_values_info(energy)

The proportion of missing values varies across the energy-generation features. Most variables contain only a very small fraction of missing entries (around 0.05% each), including `generation_biomass`, `generation_fossil_gas`, `generation_solar`, `generation_wind_onshore`, and many others.
A few features have slightly higher but still low missing rates, such as `generation_nuclear` at 0.048% and `total_load_actual` at 0.103%.

Two variables — `generation_hydro_pumped_storage_aggregated` and `forecast_wind_offshore_eday_ahead` — have 100% missing values, meaning they contain no usable data.

In [ ]:
# skipping the NaNs
energy_features = [
    f for f in energy_features
    if energy[f].notna().sum() > 0
]

In [ ]:
missing_values_info(weather)

There are no missing values in `weather` dataset.

#### 2.4.2 Identify outliers

Based on dataframes information in paragraph `2.2` there might be potential outliers such as values at the extreme ends of the distributions (e.g., very high generation levels or unusually low/high prices). These points can disproportionately influence analysis and model results. That's why now we will identify outliers.



Rolling Window Analysis: calculate rolling mean and standard deviation. Values
that deviate significantly from the rolling mean (e.g., beyond ±3 standard
deviations) may be considered outliers


##### 2.4.2.1 Target variable

In [ ]:
plt.figure(figsize=(6,5))
plt.boxplot(
    energy[target_col],
    vert=True,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', color='blue'),
    medianprops=dict(color='red'),
    whiskerprops=dict(color='blue'),
    capprops=dict(color='blue'),
    flierprops=dict(marker='o', markersize=4, markerfacecolor='blue')
)
plt.title("Boxplot of Day-Ahead Electricity Price")
plt.ylabel("Price (€/MWh)")
plt.grid(True)
plt.show()


The boxplot shows clear price spikes, confirming the presence of outliers in the target variable. These extreme values reflect real market volatility but may negatively affect several forecasting models. Therefore, while they are kept in the dataset, they will be handled later in paragraph `3.4.3` through appropriate preprocessing to minimise their impact on model performance.

##### 2.4.2.2 Energy features

In [ ]:
def boxplots(numeric_features, df):
    num_plots = len(numeric_features)
    cols = 2
    rows = math.ceil(num_plots / cols)

    plt.figure(figsize=(cols * 5, rows * 4))

    for i, feature in enumerate(numeric_features):
        plt.subplot(rows, cols, i + 1)
        sns.boxplot(y=df[feature])
        plt.title(f"Boxplot: {feature}")

    plt.tight_layout()
    plt.show()

def outliers_detection(numericFeatures, df):
    Q1 = df[numericFeatures].quantile(0.25)
    Q3 = df[numericFeatures].quantile(0.75)
    IQR = Q3 - Q1

    outliers = ((df[numericFeatures] < (Q1 - 1.5 * IQR)) | (df[numericFeatures] > (Q3 + 1.5 * IQR)))
    print("Number of outliers per numeric feature:")
    print(outliers.sum())

    outliers_cols = outliers.any()
    outliers_cols = outliers_cols[outliers_cols].index.tolist()
    print("\nColumns containing outliers:")
    print(outliers_cols)


In [ ]:
boxplots(energy_features, energy)
outliers_detection(energy_features, energy)

We performed outliers detection on all numerical features using the Interquartile Range (IQR) method and visualized it using boxplots.

Features such as `generation_fossil_gas`, `generation_hydro_pumped_storage_consumption` and `generation_other` contain a high number of outlier values.
`generation_wind_onshore`, `generation_waste`,`generation_hydro_water_reservoir` and `generation_fossil_oil` also contained a noticeable number out outliers.
`generation_biomass` contains a few outliers.
The rest of the features of `energy` dataset showed no outliers according to the IQR method.

##### 2.4.2.3 Weather features

In [ ]:
boxplots(weather_features, weather)
outliers_detection(weather_features, weather)

`pressure` and `wind_speed` contain a noticeable number of outliers.

In [ ]:
outliers_cols = ['pressure', 
                 'wind_speed', 
                 'generation_fossil_gas', 
                 'generation_hydro_pumped_storage_consumption', 
                 'generation_other', 
                 'generation_wind_onshore', 
                 'generation_waste',
                 'generation_hydro_water_reservoir',
                 'generation_biomass',
                 'generation_fossil_oil' ]

#### 2.4.3 Check data duplication

In [ ]:
energy.index.duplicated().sum()


In [ ]:
weather.index.duplicated().sum()

There are no duplicates.

---
## 3. Data Preparation

### 3.1 Data Cleaning

#### 3.1.1 Missing values

The first thing to be perfomed is to handle missing values indicated in paragraph `2.4.1`.

The missing values cannot be romoved that's why the NaNs will be filled in and columns only with NaNs will be dropped.

In [ ]:
print("Remaining NaNs in energy:", energy.isna().sum().sum())

# Drop columns that contain only NaN values
energy = energy.dropna(axis=1, how='all')

# Interpolate missing values (time-series aware)
energy.index = pd.to_datetime(energy.index, errors='coerce', utc=True)
energy = energy.interpolate(method='time')

# Forward-fill remaining NaNs (edge cases)
energy = energy.ffill().bfill()

print("Remaining NaNs in energy:", energy.isna().sum().sum())


#### 3.1.2 Handling outliers

To remove outliers identified in paragraph `2.4.2` we will perform winsorization and use **Robust Scaler** during `Data Transformation` in paragraph `3.3`.


### 3.2 Bivariate analysis

Here we will skip features that consist only of values 0.

In [ ]:
df_analysis = energy.join(weather, how="inner")

# remove columns that are entirely zero
non_zero_cols = df_analysis.columns[(df_analysis != 0).any()]

# keep only numeric features
num_df = df_analysis[non_zero_cols].select_dtypes(include="number")

# compute correlation with price
corr = num_df.corr()[target_col].drop(labels=[target_col]).sort_values()

plt.figure(figsize=(8,12))
corr.plot(kind="barh")
plt.title("Correlation with Day-Ahead Price")
plt.xlabel("Correlation")
plt.show()


The diagram shows that some variables are correlated positively, some nagetively and some are only slightly or not correlated with the target variable at all.

#### Correlation between the eletricity price and the day of the week

In [ ]:
# print(type(energy.index))
# print(energy.index[:5])

weekday_price = energy['price_day_ahead'].groupby(energy.index.to_series().dt.dayofweek).mean()
print(weekday_price)

plt.figure(figsize=(6,4))
weekday_price.plot(kind='bar')
plt.title("Average Day-Ahead Electricity Price by Day of the Week")
plt.ylabel("Price (€/MWh)")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.show()


There's no strong dependency between the day of the week and the eletricity price, however the price tends to be lower on the weekends.

##### Correlation between the temperature and the eletricity price

In [ ]:
x = df_analysis["temperature"]
y = df_analysis[target_col]

# smooth curve
low = lowess(y, x, frac=0.15)

plt.figure(figsize=(10,6))
plt.scatter(x, y, s=5, alpha=0.1)
plt.plot(low[:,0], low[:,1], linewidth=3)

plt.title("LOWESS Smoothed Relationship: Temperature vs Price")
plt.xlabel("Temperature (K)")
plt.ylabel("Day-Ahead Price (€/MWh)")
plt.grid(True)
plt.show()


There's no significant correlation between the eletricity price and the temperature.

### 3.3 Data Transformation

Since all the features are numerical they don't have to be encoded in any way.

#### 3.3.1 Dropping unnecessary variables

There are variables with all 0 values. They will be dropped.

In [ ]:
df = energy.join(weather, how="inner")

dfML = df.copy()

zero_cols = dfML.columns[(dfML == 0).all()]
print("Dropped columns with all zeros:", list(zero_cols))

dfML = dfML.drop(columns=zero_cols)

dfML.info()


#### 3.3.1 Handling variables with skewed distribution

Since not all of the skewed distrubutions concern variables with positive values we will use **Yeo-Johnson** scaler.

In [ ]:
# Cast to float before transformation to avoid dtype issues
dfML = dfML.copy()

for col in skewed_cols:
    dfML[col] = dfML[col].astype(float)

In [ ]:
pt = PowerTransformer(method='yeo-johnson')

print(df[skewed_cols].dtypes)

dfML.loc[:, skewed_cols] = pt.fit_transform(dfML.loc[:, skewed_cols])

#### 3.3.2 Handling outliers

As indicated in paragraph `3.1`, we will use **Robust Scaler** to handle outliers.

In [ ]:
# Feature scaling

# Initialize the scaler
scaler = RobustScaler()

print(outliers_cols)
print(type(outliers_cols))

# Fit the scaler on the numeric features and transform
dfML.loc[:,outliers_cols] = scaler.fit_transform(dfML[outliers_cols])

### 3.4 Data Splitting

To evaluate the models that will be used in paragraph `4` we have to split dataset into a train and a test set. We will use an 75/25 split ratio (the whole dataset represents 4 years of data and the test set will consist of last 12 months) to ensure sufficient amount of data for training while retaining enough samples for meaningful evaluation.

In [ ]:
# Last 12 months for test
split_date = dfML.index.max() - pd.DateOffset(years=1)

train_dfML = dfML[dfML.index <= split_date]
test_dfML  = dfML[dfML.index > split_date]

print("Split date:", split_date)
print("Train:", train_dfML.shape)
print("Test:", test_dfML.shape)

### 3.5 Feature selection

#### Embedded method - Lasso regularization

In [ ]:
def lasso_regularization(df, target_col="price_day_ahead"):

    X = df.drop(columns=[target_col]).copy()
    y = df[target_col].copy()

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    lasso = Lasso(alpha=0.01, random_state=42, max_iter=5000)

    sel_ = SelectFromModel(lasso)
    sel_.fit(X_scaled, y)

    selected_feat = X.columns[sel_.get_support()]
    removed_feat = X.columns[~sel_.get_support()]

    print(f"Selected features ({len(selected_feat)}):")
    print(selected_feat.tolist())

    print(f"\nRemoved features ({len(removed_feat)}):")
    print(removed_feat.tolist())

    return selected_feat

In [ ]:
Lasso_SelectedColumns = lasso_regularization(train_dfML)

Lasso_SelectedColumns

In [ ]:
train_dfML = train_dfML[list(Lasso_SelectedColumns) + [target_col]]
test_dfML  = test_dfML[list(Lasso_SelectedColumns) + [target_col]]

:asso regularization removed DOK

We prepared dataframes for future testing and tuning to compare different models and select the one with the best performance.

---
## 4. Modelling

### 4.1 Statistical Models

For the statistical modelling we will use SARIMA and SARIMAX models in order to predict future electricity prices. Since these methods are computationally heavy when applied to long hourly time series, a subset of the data covering two consecutive years will be selected. This period needs to be sufficiently long to capture the most relevant seasonal patterns, while keeping the computational cost manageable.

#### 4.1.1 Subset selection

In order to select the appropriate period for further analysis, we visualized subsequent pairs of years: 2015-2016 and 2016-2017. The year 2018 was not taken into account due to its role as a test set.

In [ ]:
test_dfML.index = pd.to_datetime(test_dfML.index)
train_dfML.index = pd.to_datetime(train_dfML.index)

periods = [
    ('2015-01-01', '2016-12-31'),
    ('2016-01-01', '2017-12-31')
]

plt.figure(figsize=(12, 8))

for i, (start, end) in enumerate(periods, 1):
    plt.subplot(2, 1, i)
    train_dfML.loc[start:end, 'price_day_ahead'].plot()
    plt.title(f'Price Day-Ahead: {start[:4]}-{end[:4]}')
    plt.grid(True)

plt.tight_layout()
plt.show()

Based on a visual assessment of subsequent annual periods, we decided to select 2016-2017 for model training. Compared to 2015-2016, this period is characterized by better consistency of the training data distribution with the 2018 test data, more stable price dynamics, and clearer seasonal patterns. However, the 2015-2016 period could also be considered a valid alternative, since it displays relevant seasonal characteristics.

In [ ]:
start = '2016-01-01'
end ='2017-12-31'
train_stat= train_dfML.loc[start:end, :]
train_stat

In [ ]:
plt.plot(train_stat['price_day_ahead'])
plt.show()

##### Decomposition

In order to better understand the underlying structure of the electricity price time series, a seasonal-trend decomposition was performed. It was applied with the use of STL method. Such approach enable us to isolate trend, seasonal, and residual components and to better assess the presence of recurring patterns.

In [ ]:
stl = STL(train_stat['price_day_ahead'], period=24, robust=True)
res = stl.fit()

fig = res.plot()
fig.suptitle(
    'STL Decomposition of Day-Ahead Electricity Price (2016-2017)',
    fontsize=14
)
plt.tight_layout()
plt.show()

Based on the original series, it can be observed that electricity prices exhibit high short-term variability and frequent price spikes, which are typical characteristics of electricity markets. The trend component indicates a moderate increase in the price level over the analysed period, providing initial evidence of non-stationarity in the data. Due to the large number of data points (hourly observations over a two-year period), the seasonal component is not clearly visible at this scale. When it comes to residual component, it does not display a clear seasonal structure. However, it contains episodes of extreme deviations, which may be associated with exceptional events affecting electricity prices.

In order to better visualise the structure of the data, and in particular the daily seasonal pattern, an additional decomposition was performed at the scale of the first month of the analysed period. This allowed for a clearer identification of recurring price patterns.

In [ ]:
short_series = train_stat['price_day_ahead'].loc['2016-01-01':'2016-01-31']

stl_short = STL(short_series, period=24, robust=True)
res_short = stl_short.fit()

fig = res_short.plot()
fig.suptitle(
    'STL Decomposition of Day-Ahead Electricity Price (January 2016)',
    fontsize=14
)
plt.tight_layout()
plt.show()


The decomposition of the time series at a monthly scale, specifically for the first month of the analysed data, allows for a much clearer identification of short-term patterns. The seasonal component shows a regular daily price cycle with repeating patterns across consecutive days. The seasonal amplitude is relatively stable, although some days exhibit stronger fluctuations. The residual component contains occasional short-term deviations, which may be related to sudden market events or random noise.

In [ ]:
res.seasonal.iloc[:72].plot(
    title='Seasonal component first 72 hours'
)
plt.show()

The chart above provides additional confirmation of the presence of a clear daily seasonal pattern, as illustrated over a three-day period.

#### 4.1.2 Stationarity testing (ADF)

In order to use the selected dataset in models such as SARIMA/SARIMAX used to predict future electricity prices, it is necessary to ensure its stationarity i.e. that its statistical properties, such as the mean, variance, and autocorrelation structure, remain constant over time. To assess whether this condition is satisfied, the Augmented Dickey-Fuller (ADF) test was performed.

It is a statistical test used to examine the presence of a unit root in a time series. The null hypothesis (H₀) states that the series contains a unit root and is therefore non-stationary. At the same time, the alternative hypothesis (H₁) assumes that the series is stationary. A sufficiently small p-value (smaller than 0.05) leads to the rejection of the null hypothesis, indicating that the data meet the criteria to be used in forcast models.

In [ ]:
def adf_test(series):
    results = adfuller(series)
    print(f"ADF statistic: {results[0]}")
    print(f"p-value: {results[1]}")
    print('Critical values: ')
    for thres, adf_stat in results[4].items():
        print(thres, adf_stat)

In [ ]:
adf_test(train_stat['price_day_ahead'])

The Augmented Dickey-Fuller test yielded a test statistic of −7.42, which is substantially lower than all reported critical values ​​at the 1%, 5%, and 10% significance levels, and a very low p-value. Although these results suggest rejection of the null hypothesis of a unit root, prior analysis of the time series indicates that the data may not meet the assumptions of stationarity. This is evidenced by the observed changes in the mean and variability over time.

Therefore, to further verify the stationarity of the series, moving statistics in the form of the moving average and moving standard deviation were analyzed. This visual inspection allows for the assessment of potential changes in the statistical properties of the series over time and provides additional confirmation of its stationarity or non-stationarity.

In [ ]:
def rolling_mean_sd(series):
    rolling_mean = series.rolling(window=168).mean()
    rolling_std = series.rolling(window=168).std()

    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    axes[0].plot(series, alpha=0.25, label='Original')
    axes[0].plot(rolling_mean, color='orange', linewidth=2, label='Rolling Mean (7 days)')
    axes[0].set_title('Rolling Mean (7 days Window)')
    axes[0].legend()
    axes[1].plot(rolling_std, color='green', linewidth=2, label='Rolling Std (7 days)')
    axes[1].set_title('Rolling Standard Deviation (7 days Window)')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
rolling_mean_sd(train_stat['price_day_ahead'])

The chart above indicates noticeable changes in the level of the rolling mean over the analysed period. Similarly, the rolling standard deviation varies over time, revealing time-varying volatility. Based on both statistics, it can be concluded that the series does not satisfy the stationarity assumptions, which contradicts the result of the ADF test.

Although the ADF test is useful in many cases, it may produce misleading results for hourly data with strong daily seasonality. The pronounced 24-hour seasonal pattern implies that observations at time t are highly correlated with those at time t − 24, which can mask non-stationary behaviour that is clearly visible in rolling statistics. Taking these observations into account, the non-stationary dataset requires appropriate transformations prior to modelling.

Therefore, seasonal differentiation with a 24-hour lag was applied to eliminate the daily seasonal component.

In [ ]:
train_seas_diff = train_stat['price_day_ahead'].diff(24).dropna()

In [ ]:
adf_test(train_seas_diff)

In [ ]:
rolling_mean_sd(train_seas_diff)

After applying seasonal differencing, the ADF test was rerun to confirm the stationarity condition. The test statistic achieved a value much lower than all critical values, and the p-value was effectively zero, leading to a clear rejection of the null hypothesis of a unit root. The stationarity is also confirmed in this case by the plots. The rolling mean oscillates around zero and no longer shows changes in its level, indicating that the daily seasonal component has been effectively removed. The rolling standard deviation also appears more stable over time compared to the original series.

In [ ]:
plt.plot(train_seas_diff.loc['2016-01-01':'2016-01-31'])

#### 4.1.3 ACF/PACF analysis

The autocorrelation function (ACF) is a key tool in time series analysis, measuring how a series correlates with lagged versions of itself and revealing patterns, trends, seasonality, and dependencies. In other words, it indicates how strongly the current value of the series is related to its past observations.

In the case of the analysed series after seasonal differencing, it can be observed that prices are still dependent on values from the previous few hours, as indicated by high ACF values at small lags (which is a typical characteristic of electricity price markets). Moreover, the gradual decline of the ACF suggests the presence of an autoregressive (AR) component in the data.

On the other hand, the lack of a sharp cutoff in the ACF indicates that a moving average (MA) component is unlikely. Additionally, the absence of pronounced seasonal peaks at lags such as 24 or 48 confirms that the daily seasonality has been effectively removed. This pattern suggests that a low-order autoregressive component should be considered in the model.

In [ ]:
plt.figure(figsize=(8,4))
plot_acf(train_seas_diff)
plt.title('ACF of price_day_ahead after seasonal differencing (lag=24)')
plt.show()

The PACF (Partial Autocorrelation Function) statistics measure the direct correlation between the current value of a time series and its past values, after removing the influence of all intermediate lags, helping to identify the order (p) of autoregressive (AR) models. The plot below shows the values ​​in each lag and the shaded blue area, which represents the 95% confidence interval. Any spikes exceeding this range indicate statistically significant partial autocorrelations.

In the analyzed time series, after seasonal differentiation, a significant spike at lag 1 (non-seasonal AR(1)) indicates that electricity prices remain strongly dependent on the previous hour, which is typical for electricity markets. Subsequent larger spikes occur around lags 24 and 48 indicate a seasonal autoregressive effect corresponding to the daily cycle, confirming the need for a seasonal AR component with a 24-hour period. Most of the remaining lags are within the confidence limits, indicating that higher-order autoregressive components are not required.

In [ ]:
plt.figure(figsize=(10, 4))
plot_pacf(
    train_seas_diff,
    lags=50,
    method='ywm'
)
plt.title('PACF of price_day_ahead after seasonal differencing (lag=24)')
plt.tight_layout()
plt.show()

#### 4.1.4 SARIMA

To implement statistical models such as SARIMA, which are needed to predict future electricity prices, it is necessary to find appropriate parameters. For this purpose, we used the results of stationarity analysis, seasonal decomposition, and plots of the autocorrelation (ACF) and partial autocorrelation (PACF) functions.

SARIMA parameters define both the non-seasonal and seasonal components of a time series model, denoted as SARIMA(p,d,q)(P,D,Q)s, where (p,d,q) are the non-seasonal orders (Autoregressive, Differencing, Moving Average), (P,D,Q) are the seasonal orders (Seasonal AR, Seasonal Difference, Seasonal MA), and s is the seasonal period length.

In the previous analysis, the need for differencing was identified due to the non-stationarity of the original time series caused by the presence of strong daily seasonality and time-varying mean and variance. For this purpose, seasonal differencing with a lag of 24 hours, corresponding to the daily cycle observed in the data, was performed. On this basis, the seasonal differencing order was set to D = 1. No need for additional non-seasonal differencing was observed, resulting in d = 0.

Inspection of the plot generated using ACF revealed that there was no sharp cutoffs and no visible seasonality. It shows that values ​​gradually decreased in subsequent lags, suggesting the absence of a dominant moving average component. As a result, the non-seasonal moving average order was set to q = 0 and the seasonal order to Q = 0.

The PACF revealed a strong spike at lag 1, indicating a short-term dependence on the previous hour and confirmed the need to include a non-seasonal autoregressive component of order p = 1. Furthermore, spikes were observed around lags 24 and 48, suggesting a seasonal autoregressive effect associated with the daily cycle. Based on this, we can conclude that a seasonal autoregressive order should be equal to P = 1.

Based on these observations, the final model specification was selected as **SARIMA(1,0,0)(1,1,0,24).**

To implement a model that would predict electricity prices both 1-step ahead and 24-step ahead, two SARIMA models were created. The first model was trained in the standard manner using the training dataset to estimate the model parameters without relying on future information. Then, a second model was defined on the combined training and test data. However, this model was not re-estimated. Instead, the parameters obtained from the first model were applied to it using the filter() method.

This two-stage approach allows for estimating model parameters based solely on training data while simultaneously generating forecasts for the entire analyzed time period. In this configuration, the model can be used both for one-step-ahead forecasts, in which the model state is updated with each new observation, and for multi-step-ahead forecasts (e.g. 24-step-ahead), where each subsequent forecast is based on previously predicted values. This solution allows for a realistic assessment of forecast quality across various time horizons.

In [ ]:
y_train = train_stat['price_day_ahead']
y_test  = test_dfML['price_day_ahead']
y_all = pd.concat([y_train, y_test])

sarima_model = SARIMAX(
    y_train,
    order=(1,0,0),
    seasonal_order=(1,1,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarima_res_train = sarima_model.fit()

sarima_model_all = SARIMAX(
    y_all,
    order=(1,0,0),
    seasonal_order=(1,1,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarima_res_all = sarima_model_all.filter(sarima_res_train.params)

sarima_res_all.plot_diagnostics(figsize=(12, 8))
plt.show()
print(sarima_res_all.summary())

Diagnostic plots were generated in order to perform an initial assessment of model adequacy and verify that no strong autocorrelation or systematic patterns remain unmodelled. The diagnostics include the standardized residuals over time, the histogram of residuals with an estimated density, the normal Q-Q plot, and the correlationogram of residuals. Overall, the diagnostic plots indicate that the chosen SARIMA specification provides a reasonable initial fit to the data and can be used later in forecasting. A more detailed analysis of the residual properties will be performed in a dedicated residual analysis section later in the report.

#### 4.1.5 SARIMAX using relevant exogenous variables


To train the SARIMAX model, a set of exogenous variables is required. In the case of the analyzed time series, the aforementioned subset was selected based on the economic relevance of individual variables and their observed correlation with electricity prices. It consists of:
- total electricity demand (total_load_actual)
- onshore wind generation (generation_wind_onshore)
- fossil gas generation (generation_fossil_gas).

The corresponding exogenous datasets were then extracted for both the training and test periods. Finally, missing-value checks were performed to ensure data consistency prior to model estimation.

In [ ]:
y_train = train_stat['price_day_ahead']
y_test  = test_dfML['price_day_ahead']
exog_cols = [
    'total_load_actual',
    'generation_wind_onshore',
    'generation_fossil_gas'
]
X_train = train_stat[exog_cols]
X_test = test_dfML[exog_cols]

print(y_train.isna().sum())
print(X_train.isna().sum())


To extend the model with additional external information, the SARIMAX model was applied. In contrast to the classical SARIMA approach, SARIMAX incorporates not only price_day_ahead data but also exogenous variables. As in the case of SARIMA, the implementation was divided into two stages.

In the first stage, the SARIMAX model was trained exclusively on the training dataset, using the corresponding exogenous variables from the same dataset. Next, a model based on the combined training dataset and test data with associated exogenous variables, was constructed. This second model was not re-estimated. Instead, the parameters obtained from the training model were reused. Diagnostic plots were also generated and, similarly to the SARIMA case, they indicate a reasonable initial fit of the model to the data.

In [ ]:
y_all = pd.concat([y_train, y_test])
X_all = pd.concat([X_train, X_test])

sarimax_model = SARIMAX(
    y_train,
    exog=X_train,
    order=(1,0,0),
    seasonal_order=(1,1,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarimax_res_train = sarimax_model.fit()

sarimax_model_all = SARIMAX(
    y_all,
    exog=X_all,
    order=(1,0,0),
    seasonal_order=(1,1,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
)
sarimax_res_all = sarimax_model_all.filter(sarimax_res_train.params)

sarimax_res_all.plot_diagnostics(figsize=(12, 8))
plt.show()
print(sarimax_res_all.summary())

#### 4.1.6 Residual diagnostics

In order to evaluate the adequacy of the time series models, we conducted a residual analysis. This step allows us to verify whether the model has successfully captured the underlying structure of the data and whether the remaining unexplained component behaves like random noise. If the residuals exhibit no systematic patterns, constant variance, and no significant autocorrelation, the model can be considered suitable for the forecasting process.

In [ ]:
residuals_sarima = sarima_res_all.resid
df_residuals = train_stat.copy()
df_residuals['residuals'] = residuals_sarima
df_residuals['fittedvales'] = sarima_res_all.fittedvalues
df_residuals[['price_day_ahead','fittedvales','residuals']]

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(residuals_sarima)
plt.axhline(0, color='black')
plt.title("Residuals over time")
plt.show()

The plot above presents the residuals of the SARIMA model over time. Most residual values are centered around zero, which indicates the absence of systematic bias and suggests that the model captures the main structure of the data reasonably well. However, several spikes can also be observed, with residuals ranging approximately from -60 to 40. These extreme deviations indicate periods where the model fails to fully explain sudden changes in electricity prices. Such anomalies are likely driven by market-specific characteristics and may be associated with exceptional events.

In [ ]:
plot_acf(residuals_sarima, lags=50)
plt.title("ACF of residuals")
plt.show()

The ACF of the SARIMA residuals shows that most autocorrelations are small and within the 95% confidence interval. However, slight peaks at lags 24 and 48 are still visible, which correspond to the daily seasonality in the hourly data. The presence of weak seasonal autocorrelation in the residuals indicates that the model captures the main dynamics of the series, but a small amount of daily dependence remains in the residuals.

In [ ]:
plt.figure(figsize=(10, 4))
plot_pacf(
    residuals_sarima,
    lags=50,
    method='ywm'
)
plt.title('PACF for residuals')
plt.tight_layout()
plt.show()

The PACF of the residuals shows that most lags are close to zero. Small spikes are visible at lags 24 and 48, consistent with the hourly structure of the data and reflecting weak, persistent daily seasonality. However, these values are small and do not justify adding additional AR terms.

In [ ]:
lb_test = acorr_ljungbox(residuals_sarima, lags=[24], return_df=True)
print(lb_test)

To examine whether autocorrelation remains in the residuals, the Ljung-Box test was applied. According to this method, the null hypothesis (H₀) of the test states that the residuals are not autocorrelated, while the alternative hypothesis indicates the presence of serial dependence. A p-value below the 0.05 leads to rejection of the null hypothesis. In case of the SARIMA model, the obtained p-value is equal to 0, which suggests the presence of residual autocorrelation. In most cases, such a result would indicate that the model may require further refinement before being used for forecasting.

However, in the context of hourly electricity price data, the Ljung-Box test may be too sensitive and produce misleading results due to the large sample size, strong seasonality and complex dependency structure of the series. The ACF and PACF plots show that the remaining autocorrelation is weak. Therefore, the Ljung-Box test result is interpreted with caution and does not invalidate the model for forecasting purposes.

##### SARIMAX exogenous variables

As with the SARIMA model, residual analysis was also performed for the SARIMAX model with exogenous variables. This analysis allows us to assess whether the residuals behave like random noise or whether further modifications to the model structure are required.

In [ ]:
residuals = sarimax_res_all.resid
df_residuals = train_stat.copy()
df_residuals['residuals'] = residuals
df_residuals['fittedvales'] = sarimax_res_all.fittedvalues
df_residuals[['price_day_ahead','fittedvales','residuals']]

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(residuals)
plt.axhline(0, color='black')
plt.title("Residuals over time")
plt.show()

The plot shows how the residual component changes over time. The residuals oscillate around zero and do not exhibit a long-term trend. However, some spikes are present, indicating that the model does not fully capture extreme price changes.

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(residuals)
plt.title("Distribution of residuals")
plt.show()

For the purpose of showing the distribution of residuals, the histogram above was created. Based on it, it can be concluded that the residuals are strongly concentrated around zero, indicating that most prediction errors are small. However, the presence of noticeable, though relatively light, tails suggests that the model occasionally struggles to capture sudden and extreme price changes, which are common in electricity markets.

In [ ]:
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q plot of residuals")
plt.show()

The Q-Q plot shows how the distribution of residuals compares to a normal distribution. While the residuals follow the reference line reasonably well in the central part, deviations appear in the tails. This indicates the presence of extreme values, which is consistent with the characteristics of electricity price markets.

In [ ]:
plot_acf(residuals, lags=50)
plt.title("ACF of residuals")
plt.show()

According to the ACF chart, most of the lag values oscillate around 0, indicating that the majority of temporal dependence has been removed by the model. However, small spikes can be observed at lags around 24 and 48, which correspond to daily seasonality in hourly data. This suggests that some seasonal structure remains in the residuals.

In [ ]:
plt.figure(figsize=(10, 4))
plot_pacf(
    residuals,
    lags=50,
    method='ywm'
)
plt.title('PACF for residuals')
plt.tight_layout()
plt.show()

The PACF of the residuals shows that most of the partial autocorrelations are close to zero. This indicates no strong, remaining direct dependencies between the residuals. Although a few spikes are visible at larger lags, (24, 48) these are relatively small and do not suggest the need for additional autoregressive terms.

In [ ]:
lb_test = acorr_ljungbox(residuals, lags=[24], return_df=True)
print(lb_test)

The Ljung-Box test was applied to the residuals at lag 24 to assess the presence of remaining autocorrelation. The very small p-value indicates that the null hypothesis of no autocorrelation should be rejected. This means that some temporal dependence may still be present in the residual. Given the complexity of the electricty market and the results of the ACF an PACF diagnostics, this outcome is interpreted as not a critical flaw of the model.

#### 4.1.7 Forecast 1-step ahead and 24-step ahead

In [ ]:
y_test  = test_dfML['price_day_ahead']

##### 1-step ahead forecast

To perform one-step-ahead forecasting using the SARIMA model, forecasts are generated sequentially for each hour of the test period. Setting dynamic=False ensures that the model's state is updated with each new observed value, closely replicating a real-time forecasting scenario in which the most recent observations are always available..

In [ ]:
pred_sarima = sarima_res_all.get_prediction(
    start=y_test.index[0],
    end=y_test.index[-1],
    dynamic=False
)

y_hat_sarima = pred_sarima.predicted_mean

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(y_test, label="True")
plt.plot(y_hat_sarima, label="SARIMA 1h-ahead")
plt.legend()
plt.show()

The generated plot shows that SARIMA forecasts largely match observed electricity prices, accurately capturing both short-term fluctuations and the overall price level. The model's greatest challenge lies in larger deviations caused by sudden price spikes or drops (typical of this market), but these differences remain within an acceptable range.

Similar to the previous model, the SARIMAX model also employs a one-step-ahead forecasting procedure, additionally incorporating exogenous variables such as total load and generation from selected energy sources. As with the SARIMA model, the model state is updated at each step using the actual observed price and its corresponding input data.

In [ ]:
pred_sarimax = sarimax_res_all.get_prediction(
    start=y_test.index[0],
    end=y_test.index[-1],
    exog=X_test,
    dynamic=False
)

y_hat_sarimax = pred_sarimax.predicted_mean

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(y_test, label="True")
plt.plot(y_hat_sarimax, label="SARIMAX 1h-ahead")
plt.legend()
plt.show()

The plot shows that the obtained results closely match the actual data from the test set. A slight improvement is visible in the case of larger deviations compared to the standard SARIMA model, indicating that the selected exogenous variables improved the model's performance.

##### 24-steps ahead

To use the SARIMA model to predict electricity prices 24 steps ahead, a rolling forecast strategy was employed. This approach works as follows: At the beginning of each test day, the model generates a forecast for the next 24 hours, without access to actual observations during this time period. This is achieved by setting dynamic=start_point, which ensures that subsequent hourly forecasts are based on previously forecasted values ​​rather than actual prices

In [ ]:
sarima_predictions_list = []

for i in range(0, len(y_test), 24):
    start_point = y_test.index[i]

    pred = sarima_res_all.get_prediction(
        start=start_point,
        end=start_point + pd.Timedelta(hours=23),
        exog=X_test,
        dynamic=start_point
    )

    sarima_predictions_list.append(pred.predicted_mean)

y_hat_24h_sarima = pd.concat(sarima_predictions_list)


In [ ]:
plt.figure(figsize=(14,4))
plt.plot(y_test, label="Real", alpha=0.6)
plt.plot(y_hat_24h_sarima, label="24-step ahead rolling (SARIMA)")
plt.legend()
plt.grid(True)
plt.show()

The resulting forecasts match the test data quite well, but it is clear that the prediction quality has deteriorated visually compared to the 1-day-ahead forecasts. They still reflect the overall level and daily structure of electricity prices. However, they exhibit greater deviations during periods of sudden price spikes or drops.

A similar method was used to test 24-step rolling forecasts using the SARIMAX model. The only difference here is the addition of exogenous data. As in SARIMA, the test data is divided into 24-hour blocks, each with a starting point, and then the corresponding 24-hour exog_block is selected from the exogenous data. Finally, a prediction is generated for the current hour to the next 23 hours.

In [ ]:
sarimax_predictions_list = []

for i in range(0, len(y_test), 24):
    start_point = y_test.index[i]

    exog_block = X_test.loc[start_point : start_point + pd.Timedelta(hours=23)]

    pred = sarimax_res_all.get_prediction(
        start=start_point,
        end=start_point + pd.Timedelta(hours=23),
        exog=exog_block,
        dynamic=start_point
    )

    sarimax_predictions_list.append(pred.predicted_mean)

y_hat_24h_sarimax = pd.concat(sarimax_predictions_list)


In [ ]:
plt.figure(figsize=(14,4))
plt.plot(y_test, label="Real", alpha=0.6)
plt.plot(y_hat_24h_sarimax, label="24-step ahead rolling (SARIMAX)")
plt.legend()
plt.grid(True)
plt.show()

The plot compares actual electricity prices with the 24-hour rolling forecast of the SARIMAX model. As with SARIMA, it shows that the model accurately reflects the overall price level and short-term dynamics, but it exhibits larger deviations compared to single-step forecasts. However, this is typical of multi-step forecasts, where the error accumulates over subsequent time horizons.

#### 4.1.8 Evaluation metrcis and comparision

In [ ]:
def compute_metrics(y_true, y_pred):
    idx = y_true.index.intersection(y_pred.index)
    y_t = y_true.loc[idx]
    y_p = y_pred.loc[idx]

    mae = np.mean(np.abs(y_t - y_p))
    rmse = np.sqrt(np.mean((y_t - y_p) ** 2))
    mape = np.mean(
        np.abs((y_t - y_p) / y_t.replace(0, np.nan))
    ) * 100

    return mae, rmse, mape

In [ ]:
results = []
results.append(('SARIMA', '1-step', *compute_metrics(y_test, y_hat_sarima)))
results.append(('SARIMAX', '1-step', *compute_metrics(y_test, y_hat_sarimax)))
results.append(('SARIMA', '24-step', *compute_metrics(y_test, y_hat_24h_sarima)))
results.append(('SARIMAX', '24-step',*compute_metrics(y_test, y_hat_24h_sarimax)))
df_stat_results = pd.DataFrame(results, columns=['Model', 'step', 'MAE', 'RMSE', 'MAPE'])
df_stat_results


To evaluate and compare the statistical models, standard error measures (MAE, RMSE, and MAPE) were calculated for each combination. The obtained results show that for both one-step-ahead and multi-step-ahead forecasts, the SARIMAX model achieves lower values ​​for all error measures than the classic SARIMA model. This indicates the benefit of including exogenous variables. Furthermore, for 24-step forecasts, the errors are significantly larger for both models, which is a natural consequence of the cumulative errors in multi-step forecasts.

In summary, both models produced satisfactory results, and the addition of exogenous variables improves the quality of forecasts.

In [ ]:
print("SARIMA AIC :", sarima_res_train.aic)
print("SARIMAX AIC:", sarimax_res_train.aic)

The displayed AIC (Akaike Information Criterion) values ​​are used to compare how well a model fits data against its complexity. In this case, the SARIMAX model achieves a significantly lower AIC than the classic SARIMA model, indicating that after including exogenous variables, the model better describes the data, despite the larger number of parameters.

### 4.2 Machine Learning Models

In this project, several machine learning regression models are trained and hyperparameter-tuned using time-series cross-validation to predict electricity prices. Their performance is then compared using error metrics in order to select the model that generalizes best for final forecasting.

#### 4.2.1 Build supervised sliding-window datasets

We will be using two functions two build sliding-window datasets. `make_sliding_window` is used for single-step predicitons and `slideWindow` for multi-step predictions. Later these functions will be used to build the sliding-window datasets after choosing the best window size in paragraph `4.2.2`.

In [ ]:
def make_sliding_window(series, window_size, horizon=1): # for single-step
    X, y = [], []
    for i in range(len(series) - window_size - horizon + 1):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size + horizon - 1])
    return np.array(X), np.array(y)


def slideWindow(time_series, window_in, horizon): # for multi-step
    if (len(time_series) - window_in - horizon + 1) <= 0:
       raise ValueError("Time series too short for window size or horizon.")
    
    d = time_series.values
    X, y = [], []

    for start in range(len(time_series)-window_in):
        end = start + window_in
        out = end + horizon
        X.append(d[start:end].reshape(-1))
        y.append(d[end:out].ravel()) # ravel() is equivalent to reshape(-1): returnscontiguous flattened array
        cols_x = [f'x{i}' for i in range(1, window_in+1)]
        cols_y = [f'y{i}' for i in range(1, horizon+1)]
        df_X = pd.DataFrame(X, columns=cols_x)
        df_y = pd.DataFrame(y, columns=cols_y)

    return pd.concat([df_X, df_y], axis=1).dropna()

def slideWindow2(time_series, window_in, horizon): # improved performance
    if (len(time_series) - window_in - horizon + 1) <= 0:
        raise ValueError("Time series too short for window size or horizon.")

    d = time_series.values
    n_samples = len(d) - window_in - horizon + 1

    X = np.empty((n_samples, window_in))
    y = np.empty((n_samples, horizon))

    for i in range(n_samples):
        X[i] = d[i : i + window_in]
        y[i] = d[i + window_in : i + window_in + horizon]

    cols_x = [f"x{i}" for i in range(1, window_in + 1)]
    cols_y = [f"y{i}" for i in range(1, horizon + 1)]

    return pd.DataFrame(
        np.hstack([X, y]),
        columns=cols_x + cols_y
    )

#### 4.2.2 Several window sizes comparison

Now we will compare three window sizes (one day, two days and one week) to find the most efficient one. For that we will use Random Forest because it's the least sensitive to data changes and variance is reduced by averaging.
To compare the results we will use MAE - Mean Aboslute Error - because it has low sensitivity.

In [ ]:
window_sizes = [24, 48, 168]   # 1 day, 2 days, 1 week
horizons = [1, 24] # the number of time steps into the future to forecast at any point in time

tscv = TimeSeriesSplit(n_splits=5)

results = []

for w in window_sizes:

    X, y = make_sliding_window(train_dfML[target_col].values, window_size=w, horizon=1)

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = RandomForestRegressor(
            n_estimators=50,
            max_depth=10,
            random_state=42,
            n_jobs=-1 # enable paralelism
        )

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)

        results.append({
            "window": w,
            "fold": fold,
            "MAE": mae
        })
        print(f"Window {w}, Fold {fold}")

results_df = pd.DataFrame(results)
print(results_df.groupby("window")["MAE"].mean())


Mean Aboslute Error (MAE) is the lowest for 24-hour long window, which means that using the previous full day of information allows the model to make more accurate predictions on average

In [ ]:
BEST_WINDOW = 24

In [ ]:
X_train, y_train = make_sliding_window(
    train_dfML[target_col].values,
    window_size=BEST_WINDOW,
    horizon=1
)

print("One Day Sliding Window Created for 1 step for the training set")

In [ ]:
X_test, y_test = make_sliding_window(
    test_dfML[target_col].values,
    window_size=BEST_WINDOW,
    horizon=1
)

print("One Day Sliding Window Created for 1 step for the testing set")

In [ ]:
df_sw_multi_step_train = slideWindow2(
        train_dfML[target_col],
        window_in=BEST_WINDOW,
        horizon=24
    )

print("One Day Sliding Window Created for 24 step for the training set")

In [ ]:
df_sw_multi_step_test = slideWindow2(
        test_dfML[target_col],
        window_in=BEST_WINDOW,
        horizon=24
    )

print("One Day Sliding Window Created for 24 step for the testing set")

#### 4.2.3 Model selection and tuning

Model selection is performed using 1-step ahead predictions, as this setting provides a fair and computationally efficient basis for comparing different algorithms. All models are trained on supervised sliding-window datasets and evaluated using time-series cross-validation, ensuring that temporal order is preserved and data leakage is avoided.

The following regression algorithms are considered: Linear Regression, Decision Tree Regressor, k-Nearest Neighbours (KNN), Support Vector Regression (SVR), Bagging Regressor, Gradient Boosting Regressor, Random Forest Regressor, XGBoost Regressor, and LightGBM Regressor. For models sensitive to feature scaling (Linear Regression, KNN, and SVR), a StandardScaler is included using a pipeline.

In [ ]:
models_stage1 = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),

    "DecisionTree": DecisionTreeRegressor(random_state=42),

    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("model", KNeighborsRegressor())
    ]),

    "SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("model", SVR())
    ]),

    "Bagging": BaggingRegressor(random_state=42),

    "GradientBoosting": GradientBoostingRegressor(random_state=42),

    "RandomForest": RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ),

    "XGBoost": XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    ),

    "LightGBM": LGBMRegressor(
        random_state=42
    )
}

In [ ]:
def safe_mape(y_true, y_pred, eps=1e-8):
    y_true = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs((y_true - y_pred) / y_true))


In [ ]:
scoring = {
    "mae": "neg_mean_absolute_error",
    "rmse": make_scorer(
        lambda y, yhat: np.sqrt(mean_squared_error(y, yhat)),
        greater_is_better=False
    ),
    "mape": make_scorer(
        safe_mape,
        greater_is_better=False
    )
}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

stage1_results = []

feature_names = [f"x{i}" for i in range(1, X_train.shape[1] + 1)]

for name, model in models_stage1.items():
    fold_mae = []
    fold_rmse = []
    fold_mape = []

    print(f"\nTraining model: {name}")

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
        print(f"  Fold {fold}/5")

        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model_fold = clone(model)

        if name == "LightGBM":
            X_tr = pd.DataFrame(X_tr, columns=feature_names)
            X_val = pd.DataFrame(X_val, columns=feature_names)

        model_fold.fit(X_tr, y_tr)
        y_pred = model_fold.predict(X_val)

        fold_mae.append(mean_absolute_error(y_val, y_pred))
        fold_rmse.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        fold_mape.append(
            np.mean(np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8)))
        )

    stage1_results.append({
        "Model": name,
        "MAE": np.mean(fold_mae),
        "RMSE": np.mean(fold_rmse),
        "MAPE": np.mean(fold_mape),
    })

    print(f"  → MAE:  {np.mean(fold_mae):.3f}")
    print(f"  → RMSE: {np.mean(fold_rmse):.3f}")
    print(f"  → MAPE: {np.mean(fold_mape):.3f}")


In [ ]:
results_stage1 = (
    pd.DataFrame(stage1_results)
      .sort_values(by=["MAE", "RMSE", "MAPE"])
      .reset_index(drop=True)
)

print("\nFinal results:")
print(results_stage1)


Before tuning:

| Model            | MAE    | RMSE   | MAPE   |
| ---------------- | ------ | ------ | ------ |
| LinearRegression | 2.0272 | 3.1191 | 0.0561 |
| RandomForest     | 2.2178 | 3.4819 | 0.0657 |
| LightGBM         | 2.2992 | 3.5899 | 0.0693 |
| GradientBoosting | 2.3319 | 3.6322 | 0.0688 |
| Bagging          | 2.3822 | 3.6588 | 0.0676 |
| XGBoost          | 2.4362 | 3.7724 | 0.0773 |
| SVR              | 3.1489 | 5.6502 | 0.1291 |
| DecisionTree     | 3.4551 | 5.2995 | 0.1020 |
| KNN              | 3.9266 | 5.5235 | 0.1241 |


*Linear Regression* clearly outperforms all other models, achieving the lowest MAE, RMSE, and MAPE, which suggests that the electricity price dynamics are largely linear for 1-step-ahead forecasting. Ensemble tree-based models (Random Forest, Gradient Boosting, LightGBM) show competitive but slightly inferior performance, while instance-based and kernel methods (KNN, SVR) perform considerably worse, indicating limited benefit from more complex non-linear models at this stage.

#### 4.2.4 Hyperparameter tuning

Hyperparameter tuning is carried out using *TimeSeriesSplit*, optimizing the Mean Absolute Error (MAE) as the primary selection criterion. This metric is chosen due to its robustness and direct interpretability in the context of electricity price forecasting. For each model, the best-performing hyperparameter configuration is selected based on cross-validation results.

Linear Regression will noe be tuned because it has no meaningful hyperparameters that control model complexity.

In [ ]:
param_grids = {
    "DecisionTree": {
        "max_depth": [3, 5, 10, None],
        "min_samples_leaf": [1, 5, 10]
    },

    "KNN": {
        "model__n_neighbors": [3, 5, 10, 20]
    },

    "SVR": {
        "model__C": [1, 10],
        "model__gamma": ["scale"]
    },

    "Bagging": {
        "n_estimators": [50, 100]
    },

    "GradientBoosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    },

    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [5, 10, None]
    },

    "XGBoost": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5]
    },

    "LightGBM": {
        "n_estimators": [200, 400],
        "learning_rate": [0.05, 0.1],
        "num_leaves": [31, 63]
    }
}


In [ ]:
tuning_results = []
best_estimators = {}

tscv = TimeSeriesSplit(n_splits=5)

for name, model in models_stage1.items():
    print(f"\nTuning model: {name}")

    if name == "LinearRegression":
        print("No tuning perfomed")
        continue

    if name in ["SVR", "LightGBM"]:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,     
            refit="mae",         
            n_jobs=1
        )
    elif name in ["RandomForest", "XGBoost"]:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,    
            refit="mae"
        )
    else:
        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grids.get(name, {}),
            cv=tscv,
            scoring=scoring,     
            refit="mae",
            n_jobs=-1
        )

    if name == "LightGBM":
        X_train_df = pd.DataFrame(
            X_train,
            columns=[f"x{i}" for i in range(1, X_train.shape[1] + 1)]
        )
        grid.fit(X_train_df, y_train)
    else:
        grid.fit(X_train, y_train)

    best_idx = grid.best_index_

    best_mae = -grid.cv_results_["mean_test_mae"][best_idx]
    best_rmse = -grid.cv_results_["mean_test_rmse"][best_idx]
    best_mape = -grid.cv_results_["mean_test_mape"][best_idx]

    tuning_results.append({
        "Model": name,
        "Best_MAE": best_mae,
        "Best_RMSE": best_rmse,
        "Best_MAPE": best_mape,
        "Best_Params": grid.best_params_
    })

    best_estimators[name] = grid.best_estimator_

    print(f"  → MAE:  {best_mae:.3f}")
    print(f"  → RMSE: {best_rmse:.3f}")
    print(f"  → MAPE: {best_mape:.3f}")


In [ ]:
results_tuning = (
    pd.DataFrame(tuning_results)
      .sort_values("Best_MAE")
      .reset_index(drop=True)
)

print("\nFinal results:")
print(results_tuning)


After tuning:

| Model            | MAE       | RMSE      | MAPE       | Best Parameters                                       |
| ---------------- | --------- | --------- | ---------- | ----------------------------------------------------- |
| **RandomForest** | **2.211** | **3.470** | **0.0658** | `max_depth=None, n_estimators=200`                    |
| Bagging          | 2.221     | 3.479     | 0.0654     | `n_estimators=100`                                    |
| LightGBM         | 2.263     | 3.544     | 0.0684     | `learning_rate=0.05, n_estimators=200, num_leaves=31` |
| GradientBoosting | 2.274     | 3.621     | 0.0703     | `learning_rate=0.05, max_depth=5, n_estimators=200`   |
| XGBoost          | 2.279     | 3.595     | 0.0707     | `learning_rate=0.05, max_depth=5, n_estimators=200`   |
| SVR              | 2.527     | 4.664     | 0.0832     | `C=10, gamma=scale`                                   |
| DecisionTree     | 2.717     | 4.049     | 0.0793     | `max_depth=10, min_samples_leaf=10`                   |
| KNN              | 3.875     | 5.434     | 0.1281     | `n_neighbors=10`                                      |


Among the evaluated machine-learning models, *Random Forest* achieved the best overall performance, obtaining the lowest MAE, RMSE, and MAPE. Ensemble methods generally outperformed single learners, while distance-based (KNN) and kernel-based (SVR) models performed worse.

***Comparison with the baseline: Seasonal Naïve***

In [ ]:
def naive_last_value_cv(X_train, y_train, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    maes, rmses, mapes = [], [], []

    for _, val_idx in tscv.split(X_train):
        X_val = X_train[val_idx]
        y_val = y_train[val_idx]

        # Naive forecast = last value in the window
        y_pred = X_val[:, -1]

        maes.append(mean_absolute_error(y_val, y_pred))
        rmses.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        mapes.append(
            np.mean(np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8)))
        )

    return {
        "Model": "Naive (Last value)",
        "MAE": np.mean(maes),
        "RMSE": np.mean(rmses),
        "MAPE": np.mean(mapes),
    }


In [ ]:
baseline_results = naive_last_value_cv(X_train, y_train)
baseline_results

In [ ]:
SEASONAL_LAG = 24
def seasonal_naive_cv(X, y, seasonal_lag=24, n_splits=5):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    maes, rmses, mapes = [], [], []

    for train_idx, val_idx in tscv.split(X):
        X_val = X[val_idx]
        y_val = y[val_idx]

        # Seasonal naive forecast
        y_pred = X_val[:, -seasonal_lag]

        maes.append(mean_absolute_error(y_val, y_pred))
        rmses.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        mapes.append(
            np.mean(
                np.abs((y_val - y_pred) / np.maximum(np.abs(y_val), 1e-8))
            )
        )

    return {
        "Model": "SeasonalNaive",
        "MAE": np.mean(maes),
        "RMSE": np.mean(rmses),
        "MAPE": np.mean(mapes),
    }

In [ ]:
baseline_result = seasonal_naive_cv(
    X_train,
    y_train,
    seasonal_lag=24,
    n_splits=5
)

baseline_df = pd.DataFrame([baseline_result])

In [ ]:
# final results table

before_df = (
    pd.DataFrame(stage1_results)
    .rename(columns={
        "MAE": "MAE_before",
        "RMSE": "RMSE_before",
        "MAPE": "MAPE_before"
    })
    [["Model", "MAE_before", "RMSE_before", "MAPE_before"]]
)

after_df = (
    pd.DataFrame(tuning_results)
    .rename(columns={
        "Best_MAE": "MAE_after",
        "Best_RMSE": "RMSE_after",
        "Best_MAPE": "MAPE_after"
    })
    [["Model", "MAE_after", "RMSE_after", "MAPE_after"]]
)

final_models = (
    before_df
    .merge(after_df, on="Model", how="left")
    .sort_values("MAE_before")
    .reset_index(drop=True)
)

baseline_row = pd.DataFrame([{
    "Model": "SeasonalNaive",
    "MAE_before": baseline_df.loc[baseline_df["Model"] == "SeasonalNaive", "MAE"].values[0],
    "RMSE_before": baseline_df.loc[baseline_df["Model"] == "SeasonalNaive", "RMSE"].values[0],
    "MAPE_before": baseline_df.loc[baseline_df["Model"] == "SeasonalNaive", "MAPE"].values[0],
    "MAE_after": np.nan,
    "RMSE_after": np.nan,
    "MAPE_after": np.nan,
}])

final_table = pd.concat(
    [final_models, baseline_row],
    ignore_index=True
)

final_table["Model"] = pd.Categorical(
    final_table["Model"],
    ordered=True
)

final_table = final_table.sort_values("Model").reset_index(drop=True)

print(final_table)


|     *Model*      | MAE_before | RMSE_before | MAPE_before | MAE_after | RMSE_after | MAPE_after |
| :--------------: | :--------: | :---------: | :---------: | :-------: | :--------: | :--------: |
| LinearRegression |  2.027226  |  3.119120   |  0.056081   |     -     |     -      |     -      |
|   RandomForest   |  2.217789  |  3.481925   |  0.065677   | 2.210768  |  3.470076  |  0.065812  |
|     Bagging      |  2.382228  |  3.658807   |  0.067565   | 2.221170  |  3.478677  |  0.065382  |
|     LightGBM     |  2.299236  |  3.589891   |  0.069324   | 2.262874  |  3.543766  |  0.068422  |
| GradientBoosting |  2.331883  |  3.632150   |  0.068788   | 2.273512  |  3.620644  |  0.070329  |
|     XGBoost      |  2.436203  |  3.772397   |  0.077259   | 2.279497  |  3.595189  |  0.070680  |
|       SVR        |  3.148904  |  5.650184   |  0.129055   | 2.527390  |  4.664315  |  0.083185  |
|   DecisionTree   |  3.455143  |  5.299461   |  0.102029   | 2.717259  |  4.048858  |  0.079340  |
|       KNN        |  3.926630  |  5.523529   |  0.124120   | 3.875492  |  5.433601  |  0.128079  |
|  ***BASELINE***  |  **MAE**   |  **RMSE**   |  **MAPE**   |           |            |            |
|  SeasonalNaive   |  7.570453  |  10.922027  |  0.236852   |     -     |     -      |     -      |

The *Seasonal Naïve* baseline performs substantially worse than all machine-learning models, confirming the value of learned approaches. Hyperparameter tuning improves most models slightly, with *Random Forest* achieving the best overall performance after tuning. *Linear Regression* performs strongly without tuning and remains competitive with ensemble methods.

In [ ]:
best_model = "LinearRegression"

#### 4.2.5 Perform predictions

After selecting the best-performing machine learning model and the optimal window size, the model is retrained on the full training dataset and used to generate forecasts on the unseen test set. Predictions are produced for both single-step (1 hour ahead) and multi-step (24 hours ahead) horizons.

##### 4.2.5.1 Single-step

In this step, the selected machine learning model is trained using the optimal sliding-window size and used to generate 1-step-ahead forecasts on the test set. The predicted electricity prices are then compared against the real observed values to assess the model’s short-term forecasting accuracy.

In [ ]:
model_1step = clone(models_stage1[best_model])

print("Training 1-step ahead model...")
model_1step.fit(X_train, y_train)

y_pred_1step = model_1step.predict(X_test)
print("1-step ahead model trained.")

##### 4.2.5.2 Multi-step (24-step)

For the 24-step ahead scenario, the model predicts the next 24 hourly prices simultaneously based on the input window. This task is more challenging, as prediction errors can accumulate across the forecasting horizon, and it assesses the model’s capability to learn daily patterns and longer-term temporal dynamics.

Since `slideWindow` function returned a dataset and not `X` and `y` values, first, these variables must be extracted from the generated dataset in paragraph `4.2.1`.

In [ ]:
X_train_24 = df_sw_multi_step_train.filter(like="x").values
y_train_24 = df_sw_multi_step_train.filter(like="y").values

X_test_24 = df_sw_multi_step_test.filter(like="x").values
y_test_24 = df_sw_multi_step_test.filter(like="y").values

model_24step = clone(models_stage1[best_model])

print("Training 24-step ahead model...")
model_24step.fit(X_train_24, y_train_24)

y_pred_24step = model_24step.predict(X_test_24)
print("24-step ahead model trained.")

#### 4.2.5.3 Plots comparing predictions vs. real test data

In [ ]:
plt.figure()
plt.plot(y_test[:200], label="Actual")
plt.plot(y_pred_1step[:200], label="Prediction")
plt.title("1-Step Ahead Prediction vs Actual")
plt.xlabel("Time")
plt.ylabel("Target")
plt.legend()
plt.show()

The 1-step ahead prediction closely follows the actual electricity price series, capturing both short-term fluctuations and larger trend changes with high accuracy. Small deviations mainly appear around sharp peaks and sudden drops, which is expected since abrupt changes are harder to predict from past values alone.

In [ ]:
plt.figure()
plt.plot(y_test_24[0], marker="o", label="Actual (24h)")
plt.plot(y_pred_24step[0], marker="x", label="Prediction (24h)")
plt.title("24-Step Ahead Forecast (First Test Window)")
plt.xlabel("Forecast Horizon (hours)")
plt.ylabel("Target")
plt.legend()
plt.show()

y_test_full = y_test_24[:, 0]
y_pred_full = y_pred_24step[:, 0]

plt.figure(figsize=(14, 5))
plt.plot(y_test_full, label="Actual")
plt.plot(y_pred_full, label="Predicted")
plt.title("24-step Model – 1-hour Ahead Predictions over Full Test Set")
plt.xlabel("Time")
plt.ylabel("Electricity Price")
plt.legend()
plt.grid(True)
plt.show()

The first plot (24-step ahead, first test window) shows poor accuracy because it is a direct multi-output forecast, where the model must predict all 24 future hours at once; errors accumulate quickly and the model struggles to capture the full daily amplitude and level.

The second plot (full test set) looks much better because it effectively evaluates many overlapping short-horizon predictions, where the model mainly follows the overall pattern and seasonality, even though individual point-wise errors are still present.

### 4.3 Deep Learning Models

TODO

LSTM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# ==========================================
# 1. Data Preparation
# ==========================================
WINDOW_SIZE = 48  # Lookback period (e.g., 48 hours)
BATCH_SIZE = 32   # Hyperparameter to tune
EPOCHS = 50       # Max epochs

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_dfML)
test_scaled = scaler.transform(test_dfML)

X_train_full, y_train_full = make_sliding_window(train_scaled, WINDOW_SIZE, horizon=1)
X_test, y_test = make_sliding_window(test_scaled, WINDOW_SIZE, horizon=1)

y_train_full = y_train_full[:, 0]
y_test = y_test[:, 0]

val_split = int(len(X_train_full) * 0.8)
X_train, y_train = X_train_full[:val_split], y_train_full[:val_split]
X_val, y_val = X_train_full[val_split:], y_train_full[val_split:]

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")

In [ ]:
# ==========================================
# 2. LSTM Model Definition
# ==========================================
def build_lstm_model(input_shape):
    lstm_model = Sequential()

    # Input Layer
    lstm_model.add(Input(shape=input_shape))

    # LSTM Layer 1
    lstm_model.add(LSTM(units=50, return_sequences=True))
    lstm_model.add(Dropout(0.2))

    # LSTM Layer 2
    lstm_model.add(LSTM(units=25, return_sequences=False))
    lstm_model.add(Dropout(0.2))

    # Output Layer
    lstm_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    lstm_model.compile(optimizer=optimizer, loss='mse')
    return lstm_model


lstm_model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

In [ ]:
# ==========================================
# 3. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = lstm_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 4. Evaluation & Metrics
# ==========================================
y_pred_scaled = lstm_model.predict(X_test)

def inverse_transform_target(pred_array, scaler, n_features):
    dummy = np.zeros((len(pred_array), n_features))
    dummy[:, 0] = pred_array.flatten()
    return scaler.inverse_transform(dummy)[:, 0]

y_pred_real = inverse_transform_target(y_pred_scaled, scaler, train_dfML.shape[1])
y_test_real = inverse_transform_target(y_test.reshape(-1, 1), scaler, train_dfML.shape[1])

mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))
non_zero_mask = y_test_real != 0
mape = np.mean(np.abs((y_test_real[non_zero_mask] - y_pred_real[non_zero_mask]) / y_test_real[non_zero_mask])) * 100

print(f"\nModel Performance:")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

plt.figure(figsize=(15, 6))
plt.plot(y_test_real[:200], label='Actual Price', color='blue')
plt.plot(y_pred_real[:200], label='Predicted Price', color='red', linestyle='--')
plt.title('LSTM Forecast: Actual vs Predicted (First 200 Hours)')
plt.legend()
plt.show()

RESULTS ANALYSIS

The Long Short-Term Memory (LSTM) network demonstrated strong performance in capturing the temporal dependencies of electricity prices, achieving an MAE of 0.0959 and an RMSE of 0.1532. The high MAPE (82.15%) is attributed to the presence of near-zero prices in the test set rather than poor predictive quality.

Visually, the LSTM predictions track the actual test data very closely, successfully replicating the daily seasonality and general trends of the market. A slight "reaction delay" was observed, which is expected in autoregressive models that rely on immediate past windows. While the model handles standard volatility well, it occasionally struggles with extreme, rapid fluctuations (drastic down-and-up movements), where it tends to smooth the curve rather than capturing the full amplitude of the sharpest spikes.

The training process was stable, with training and validation loss curves converging naturally. The EarlyStopping mechanism was not triggered, and the optimal model weights were recorded at epoch 49 (out of 50). This continuous improvement throughout the training cycle indicates that the model had appropriate capacity and learning rate settings for this dataset, avoiding both premature convergence and overfitting.

GRU

In [ ]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam

In [ ]:
# ==========================================
# 1. GRU Model Definition
# ==========================================
def build_gru_model(input_shape):
    gru_model = Sequential()

    # Input Layer
    gru_model.add(Input(shape=input_shape))

    # GRU Layer 1
    gru_model.add(GRU(units=50, return_sequences=True))
    gru_model.add(Dropout(0.2))

    # GRU Layer 2
    gru_model.add(GRU(units=25, return_sequences=False))
    gru_model.add(Dropout(0.2))

    # Output Layer
    gru_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    gru_model.compile(optimizer=optimizer, loss='mse')
    return gru_model


gru_model = build_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))

In [ ]:
# ==========================================
# 2. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history_gru = gru_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)

plt.figure(figsize=(10, 6))
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('GRU Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 3. Evaluation
# ==========================================
y_pred_scaled_gru = gru_model.predict(X_test)

y_pred_real_gru = inverse_transform_target(y_pred_scaled_gru, scaler, train_dfML.shape[1])

mae_gru = mean_absolute_error(y_test_real, y_pred_real_gru)
rmse_gru = np.sqrt(mean_squared_error(y_test_real, y_pred_real_gru))
non_zero_mask = y_test_real != 0
mape_gru = np.mean(np.abs((y_test_real[non_zero_mask] - y_pred_real_gru[non_zero_mask]) / y_test_real[non_zero_mask])) * 100

print(f"\nGRU Model Performance:")
print(f"MAE:  {mae_gru:.4f}")
print(f"RMSE: {rmse_gru:.4f}")
print(f"MAPE: {mape_gru:.4f}%")

plt.figure(figsize=(15, 6))
plt.plot(y_test_real[:200], label='Actual Price', color='black', alpha=0.7)
plt.plot(y_pred_real[:200], label='LSTM Prediction', color='red', linestyle='--', alpha=0.7)
plt.plot(y_pred_real_gru[:200], label='GRU Prediction', color='green', linestyle='-.', alpha=0.7)
plt.title('Comparison: LSTM vs GRU vs Actual (First 200 Hours)')
plt.legend()
plt.show()

GRU Analysis

The Gated Recurrent Unit (GRU) model outperformed the LSTM on this dataset, achieving a lower MAE of 0.0822 (vs. 0.0959 for LSTM) and RMSE of 0.1381 (vs. 0.1532 for LSTM). This indicates that the GRU provided a slightly more accurate fit to the test data overall. As expected, the training time was shorter, 14 minutes insted of 18 for LSTM.

Visually, the GRU predictions followed a very similar trajectory to the LSTM, closely tracking the actual price curve. This suggests that both recurrent architectures successfully captured the underlying temporal patterns of the electricity market. The fact that the simpler GRU architecture yielded better metrics suggests that the additional complexity of the LSTM (extra gates and separate cell state) was not necessary for this specific 1-step forecast task and may have introduced slight inefficiencies or noise.

Similar to the LSTM, the GRU showed stable learning and continued to improve throughout the training process, with the best model weights saved at the final epoch 50. This confirms that the model did not overfit and could potentially benefit from an even longer training duration, though the current performance is already superior to the LSTM baseline.

24-Step Ahead Model

In [ ]:
# ==========================================
# 1. Data Preparation for 24-Step Ahead
# ==========================================
def create_24h_window(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size - 24 + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size: i + window_size + 24, 0])
    return np.array(X), np.array(y)

WINDOW_SIZE = 48

X_train_24, y_train_24 = create_24h_window(train_scaled, WINDOW_SIZE)
X_test_24, y_test_24 = create_24h_window(test_scaled, WINDOW_SIZE)

val_split_24 = int(len(X_train_24) * 0.8)
X_train_sub_24, y_train_sub_24 = X_train_24[:val_split_24], y_train_24[:val_split_24]
X_val_24, y_val_24 = X_train_24[val_split_24:], y_train_24[val_split_24:]

print(f"Input Shape: {X_train_sub_24.shape}")
print(f"Target Shape: {y_train_sub_24.shape}")

In [ ]:
# ==========================================
# 2. LSTM Model for 24-Step Output
# ==========================================
model_24 = Sequential()
# Input Layer
model_24.add(Input(shape=(X_train_sub_24.shape[1], X_train_sub_24.shape[2])))

# LSTM24 Layer 1
model_24.add(LSTM(128, return_sequences=True))
model_24.add(Dropout(0.2))

# LSTM24 Layer 2
model_24.add(LSTM(68, return_sequences=False))
model_24.add(Dropout(0.2))

# OUTPUT LAYER
model_24.add(Dense(24))

model_24.compile(optimizer='adam', loss='mse')

In [ ]:
# ==========================================
# 3. Training
# ==========================================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history_24 = model_24.fit(
    X_train_sub_24, y_train_sub_24,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_24, y_val_24),
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# ==========================================
# 4. Evaluation
# ==========================================
y_pred_scaled_24 = model_24.predict(X_test_24)

n_samples = y_pred_scaled_24.shape[0]
n_steps = 24
n_features = train_dfML.shape[1]

flat_preds = y_pred_scaled_24.reshape(-1, 1)
dummy_preds = np.zeros((len(flat_preds), n_features))
dummy_preds[:, 0] = flat_preds.flatten()
flat_real_preds = scaler.inverse_transform(dummy_preds)[:, 0]

y_pred_real_24 = flat_real_preds.reshape(n_samples, n_steps)

flat_actuals = y_test_24.reshape(-1, 1)
dummy_actuals = np.zeros((len(flat_actuals), n_features))
dummy_actuals[:, 0] = flat_actuals.flatten()
flat_real_actuals = scaler.inverse_transform(dummy_actuals)[:, 0]
y_test_real_24 = flat_real_actuals.reshape(n_samples, n_steps)

mae_24 = mean_absolute_error(y_test_real_24, y_pred_real_24)
rmse_24 = np.sqrt(mean_squared_error(y_test_real_24, y_pred_real_24))
mask_24 = y_test_real_24 != 0
mape_24 = np.mean(np.abs((y_test_real_24[mask_24] - y_pred_real_24[mask_24]) / y_test_real_24[mask_24])) * 100

print(f"\n24-Step Ahead Model Performance:")
print(f"MAE:  {mae_24:.4f}")
print(f"RMSE: {rmse_24:.4f}")
print(f"MAPE: {mape_24:.4f}%")


sample_idx = 0
plt.figure(figsize=(10, 5))
plt.plot(range(24), y_test_real_24[sample_idx], marker='o', label='Actual Day')
plt.plot(range(24), y_pred_real_24[sample_idx], marker='x', linestyle='--', label='Predicted Day')
plt.title(f'24-Hour Forecast for Sample #{sample_idx}')
plt.xlabel('Hour of Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 5. Continuous Plot (Day-Ahead Simulation)
# ==========================================

stitched_pred = y_pred_real_24[::24].flatten()
stitched_actual = y_test_real_24[::24].flatten()

plot_hours = 240 # 10 days

plot_hours = min(plot_hours, len(stitched_pred))

plt.figure(figsize=(15, 6))

plt.plot(stitched_actual[:plot_hours], label='Actual Price', color='blue', alpha=0.6)

plt.plot(stitched_pred[:plot_hours], label='Day-Ahead Prediction (Stitched)',
         color='orange', linestyle='--', marker='.', markersize=5)

plt.title(f'Day-Ahead Forecast: Continuous View (First {plot_hours} Hours)')
plt.xlabel('Hours')
plt.ylabel('Electricity Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

mae_stitched = mean_absolute_error(stitched_actual, stitched_pred)
print(f"Realistic Day-Ahead Operation MAE: {mae_stitched:.4f}")

24-Step LSTM Analysis

Predicting a full 24-hour horizon proved to be significantly more challenging than the 1-step task. The model achieved an MAE of 0.2650 and RMSE of 0.3563. These errors are roughly 3x higher than the 1-step GRU/LSTM models (MAE ~0.08), reflecting the inherent difficulty of forecasting an entire day without the ability to correct course using intermediate real values.

On a larger scale, the model demonstrates a basic ability to follow the general market trend, confirming it learned the overall weekly and daily seasonality. A major limitation observed is a substantial "phase shift" or lag. When a rapid price drop occurs, the model often predicts this drop only after the event has passed (likely with a 24-hour delay). This suggests the model is relying heavily on persistence (repeating yesterday's pattern) rather than anticipating changes based on exogenous features like weather. Visually, the predictions are frequently "off" from the actual values, lacking the precision seen in the 1-step models. This is consistent with the EarlyStopping occurring at epoch 15 (best weights at epoch 5), indicating the model struggled to find a complex pattern and converged quickly to a simpler, safer suboptimal solution.

Hyperparameter tuning

In [ ]:
batch_sizes = [16, 32, 64]
results = {}

for bs in batch_sizes:
    print(f"Testing Batch Size: {bs}")
    model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    # Train
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=bs,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=0
    )

    best_val_loss = min(history.history['val_loss'])
    results[bs] = best_val_loss
    print(f"Batch Size {bs} - Best Val Loss: {best_val_loss:.5f}")

print("Best Configuration:", min(results, key=results.get))

In [ ]:
def build_tunable_lstm(input_shape, units, num_layers, dropout_rate=0.2):
    tunable_lstm_model = Sequential()

    # Input Layer
    tunable_lstm_model.add(Input(shape=input_shape))

    # First Layer (Always present)
    return_seq = (num_layers > 1)
    tunable_lstm_model.add(LSTM(units=units, return_sequences=return_seq))
    tunable_lstm_model.add(Dropout(dropout_rate))

    # Second Layer (Conditional)
    if num_layers == 2:
        tunable_lstm_model.add(LSTM(units=units // 2, return_sequences=False))
        tunable_lstm_model.add(Dropout(dropout_rate))

    # Output Layer
    tunable_lstm_model.add(Dense(1))

    optimizer = Adam(learning_rate=0.0001)
    tunable_lstm_model.compile(optimizer=optimizer, loss='mse')
    return tunable_lstm_model

In [ ]:
# ==========================================
# Hyperparameter Tuning: Units & Layers
# ==========================================

units_options = [32, 64, 128]
layers_options = [1, 2]

tuning_results = []

print("Starting Hyperparameter Tuning...")
print("-" * 50)

for units in units_options:
    for layers in layers_options:
        print(f"Testing: {units} Units | {layers} Layers...")

        model = build_tunable_lstm(
            input_shape=(X_train.shape[1], X_train.shape[2]),
            units=units,
            num_layers=layers
        )

        history = model.fit(
            X_train, y_train,
            epochs=30,
            batch_size=32, # Using your fixed best batch size
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=0
        )

        best_val_loss = min(history.history['val_loss'])

        tuning_results.append({
            'units': units,
            'layers': layers,
            'val_loss': best_val_loss
        })

        print(f"   -> Result: Best Val Loss = {best_val_loss:.5f}")

print("-" * 50)
print("Tuning Complete.")

In [ ]:
results_df = pd.DataFrame(tuning_results)

results_df = results_df.sort_values(by='val_loss', ascending=True)

print("\nTop 3 Configurations:")
print(results_df.head(3))

best_config = results_df.iloc[0]
print(f"\nWINNER: LSTM with {int(best_config['units'])} Units and {int(best_config['layers'])} Layers.")

Hyperparameter Analysis

The tuning process revealed a clear preference for a simpler architecture. The configuration with 32 units achieved the lowest validation loss (0.000856), outperforming larger configurations like 128 units (0.000904) or 64 units (0.000981). The differences are extremly slow and multiple test showed that the results may vary depending on test.

The fact that 1-layer models consistently outperformed 2-layer models (which did not appear in the top 3) suggests that the features in this dataset (price, weather, generation) are strong enough that a deeper, more complex network is not required. Adding a second layer likely introduced unnecessary noise or optimization difficulties rather than learning new abstractions.

Selecting the 32-unit model not only improves accuracy but also significantly reduces the computational cost, making the model faster to train and lighter to deploy.

In [ ]:
# ==========================================
# Seasonal Naive
# ==========================================
seasonality = 168
y_naive_pred = []
y_naive_real = []

test_values = test_dfML.iloc[:, 0].values # Assuming price is col 0

for i in range(seasonality, len(test_values)):
    y_naive_pred.append(test_values[i - seasonality])
    y_naive_real.append(test_values[i])

mae_naive = mean_absolute_error(y_naive_real, y_naive_pred)
rmse_naive = np.sqrt(mean_squared_error(y_naive_real, y_naive_pred))
print(f"Baseline (Seasonal Naive) - MAE: {mae_naive:.4f}, RMSE: {rmse_naive:.4f}")

plt.figure(figsize=(15, 6))
plt.plot(y_naive_real[:240], label='Actual Price', color='tab:blue', alpha=0.7)
plt.plot(y_naive_pred[:240], label='10-Day Recursive Forecast', color='tab:orange', linestyle='--')

plt.title(f'10-Day Recursive Forecast (LSTM)')
plt.ylabel('Price (€)')
plt.xlabel('Hours (0 to 240)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

Naive Analysis

The Seasonal Naive baseline performed significantly worse than the Deep Learning models, yielding an MAE of 0.4179 and RMSE of 0.5468. In comparison, the best-performing model (GRU) achieved an MAE of roughly 0.08, representing an improvement of approximately 80% over this baseline. This huge gap confirms that the project's complexity was justified.

The graph appears to "predict something" in certain areas because electricity demand does have a strong weekly structure. The areas where the prediction looks "random" or fails completely represent times when exogenous factors (weather changes, wind generation, holidays) shifted the price away from the historical average. Because the Naïve model blindly copies the previous week's value , it cannot account for these dynamic changes, leading to large errors whenever the weather pattern changes from one week to the next.

The poor performance of the Seasonal Naïve model demonstrates that electricity prices are not merely repetitive cycles. They are driven by complex, non-linear dependencies on weather and generation data, which only the Deep Learning models (LSTM/GRU) were able to successfully learn and predict.

---
## 6. Conclusions and Recommendations

### 6.1 Summary of Findings


### 6.2 Best Model Selection

### 6.3 Future Work


### 6.4 Lessons Learned


---

## Project Notes and Team Collaboration

### Team Members

- Julia Kardasz 1250264
- Mateusz Nowak 1250296
- Emilia Pawlowska 1250230